In [82]:
import pandas as pd
import numpy as np
import sklearn
import statsmodels

In [83]:
!python --version
print(pd. __version__)
!jupyter --version
print(sklearn.__version__)
print(statsmodels.__version__)

Python 3.11.7
2.1.4
Selected Jupyter core packages...
IPython          : 8.20.0
ipykernel        : 6.28.0
ipywidgets       : 7.7.5
jupyter_client   : 7.3.2
jupyter_core     : 5.5.0
jupyter_server   : 2.10.0
jupyterlab       : 4.0.13
nbclient         : 0.8.0
nbconvert        : 7.10.0
nbformat         : 5.9.2
notebook         : 6.4.8
qtconsole        : 5.4.2
traitlets        : 5.7.1
1.2.2
0.14.0


## Reading CSV's

In [86]:
cannabis_df = pd.read_csv('/Users/matthew/Downloads/CleanedAphasiaData - Sheet1-2.csv')
print(cannabis_df.head)

ndf =  pd.read_csv('/Users/matthew/Downloads/NormAphasiaData.csv')
ndf


<bound method NDFrame.head of     Cannabis Induced  Aphasia  Dysarthria
0                  1        0           0
1                  1        0           0
2                  1        0           0
3                  1        0           0
4                  1        1           0
5                  1        0           1
6                  1        0           0
7                  1        0           0
8                  1        0           1
9                  1        1           0
10                 1        0           0
11                 1        1           0
12                 1        0           1
13                 1        1           0
14                 1        0           1
15                 1        0           1
16                 1        0           1
17                 1        0           0
18                 1        1           0
19                 1        0           0
20                 1        0           0
21                 1        0           1
22  

,Baseline Descriptive(Row % where denominator is corresponding number with an assessment),Unnamed: 1,"Aphasia presenta n = 37,537(41)","Dysarthria presentan = 46,330(52)","Neither presentbn = 31,824(36)","Only aphasia presentbn = 11,202(12)","Only dysarthria presentbn = 21,058(24)","Both aphasia & dysarthria presentbn = 24,890(28)"
0,NaN,NaN,Included if NIHSS Aphasia completed(row%),Included if NIHSS Dysarthria completed(row%),Includes only those with both NIHSS aphasia an...,NaN,NaN,NaN
1,Age,Mean[S.D],78[13],77[13],74[14],77[13],75[13],78[13]
2,Min;Max,18;114,18;114,18;112,18;114,18;114,18;112,NaN
3,Gender,Female,20481(44),23895(52),15927(35),6049(13),10143(22),13544(30)
4,Male,17056(39),22435(52),15897(37),5153(12),10915(25),11346(26),NaN
5,Pre-stroke mRS,Independent ≤2,28218(40),35914(50),26988(38),8867(12),17099(24),18312(26)
6,Dependent >2,8819(48),7209(40),4836(27),2335(13),3959(22),6578(37),NaN
7,"No. Co-morbiditiesOverall:Med = 1,IQR = 1,2",0,8233(38),10574(49),8265(39),2509(12),5066(24),5416(25)
8,1 to 2,23867(42),29345(52),19930(35),7158(13),13299(24),15811(28),NaN
9,3 to 5,5437(46),6411(55),3629(31),1535(13),2693(23),3663(32),NaN


## Create Control Group

In [57]:
# Define the number of samples
num_samples = 1000

# Create an empty DataFrame
baseline_df = pd.DataFrame()

# Set 'Cannabis Induced' to 0 for all rows
baseline_df['Cannabis Induced'] = [0] * num_samples

# Proportions based on provided summary (convert these to probabilities)
# Assuming n = 91,801 based on the summary data provided
total_patients = 91801

# Extracting proportions (counts) from the given summary statistics
# These are approximate values extracted from your provided data
aphasia_present = 37537
dysarthria_present = 46330

# Calculate probabilities
prob_aphasia = aphasia_present / total_patients
prob_dysarthria = dysarthria_present / total_patients

# Generate random data for 'Aphasia Present'
baseline_df['Aphasia'] = np.random.choice([0, 1], size=num_samples, p=[1 - prob_aphasia, prob_aphasia])

# Generate random data for 'Dysarthria Present'
baseline_df['Dysarthria'] = np.random.choice([0, 1], size=num_samples, p=[1 - prob_dysarthria, prob_dysarthria])

# Display the first few rows of the created DataFrame
print(baseline_df)

# # Save the synthetic dataset to a CSV file (optional)
baseline_df.to_csv('synthetic_baseline_dataset.csv', index=False)


     Cannabis Induced  Aphasia  Dysarthria
0                   0        0           0
1                   0        1           0
2                   0        1           0
3                   0        0           0
4                   0        1           1
..                ...      ...         ...
995                 0        0           0
996                 0        0           1
997                 0        1           1
998                 0        1           1
999                 0        0           1

[1000 rows x 3 columns]


In [89]:
print(baseline_df.describe(include='all'))

       Cannabis Induced      Aphasia   Dysarthria
count            1000.0  1000.000000  1000.000000
mean                0.0     0.381000     0.477000
std                 0.0     0.485876     0.499721
min                 0.0     0.000000     0.000000
25%                 0.0     0.000000     0.000000
50%                 0.0     0.000000     0.000000
75%                 0.0     1.000000     1.000000
max                 0.0     1.000000     1.000000


## Merge datasets

In [90]:
merged_df = pd.concat([baseline_df, cannabis_df], ignore_index=True)
merged_df
merged_df.to_csv('Merged_df.csv', index=False)

## Logistic Regression Aphasia

In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import statsmodels.api as sm

X = merged_df[['Cannabis Induced']]
y = merged_df['Aphasia']

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Print the summary
print(result.summary())

# Interpret the coefficients
coefficients = result.params
print("\nCoefficients:")
print(coefficients)

# Odds ratios
odds_ratios = np.exp(coefficients)
print("\nOdds Ratios:")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.661936
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                Aphasia   No. Observations:                 1039
Model:                          Logit   Df Residuals:                     1037
Method:                           MLE   Df Model:                            1
Date:                Wed, 22 May 2024   Pseudo R-squ.:                0.001182
Time:                        23:20:00   Log-Likelihood:                -687.75
converged:                       True   LL-Null:                       -688.57
Covariance Type:            nonrobust   LLR p-value:                    0.2020
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.4853      0.065     -7.453      0.000      -0.613      -0.358
Cannabis In

## Logistic Regression Dysarthria

In [92]:
X = merged_df[['Cannabis Induced']]
y = merged_df['Dysarthria']

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Print the summary
print(result.summary())

# Interpret the coefficients
coefficients = result.params
print("\nCoefficients:")
print(coefficients)

# Odds ratios
odds_ratios = np.exp(coefficients)
print("\nOdds Ratios:")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.689279
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             Dysarthria   No. Observations:                 1039
Model:                          Logit   Df Residuals:                     1037
Method:                           MLE   Df Model:                            1
Date:                Wed, 22 May 2024   Pseudo R-squ.:                0.003100
Time:                        23:20:02   Log-Likelihood:                -716.16
converged:                       True   LL-Null:                       -718.39
Covariance Type:            nonrobust   LLR p-value:                   0.03482
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0921      0.063     -1.454      0.146      -0.216       0.032
Cannabis In